In [1]:
import pandas as pd

In [2]:
from src.parsers.moyka78_page import Moyka78Page
from src.parsers.utils import download_url

In [3]:
s = open('./raw/links/links_moika78.txt').read()

is_link = lambda word: word.startswith('https')
filter_link = lambda row: list(filter(is_link, row.split()))

links = list(map(filter_link, s.splitlines()))
links = list(filter(lambda arr: len(arr), links))
links = [x[0] for x in links]
links[:3]

['https://moika78.ru/news/2018-01-23/12164-invalidam-perekryli-vkhody-v-kafe-i-magaziny-kronshtadta/',
 'https://moika78.ru/news/2018-01-30/13170-peterburgskiey-invalidy-ne-khotyat-khvalit-metropoliten/',
 'https://moika78.ru/news/2018-01-31/13312-peterburgskikh-roditeley-detey-invalidov-podderzhat-spetsialisty-goryachey-linii/']

In [4]:
from time import sleep
from tqdm import tqdm

records = []

for i, link in tqdm(enumerate(links), total=len(links)):
    sleep(0.2)
    
    try:
        source = download_url(link)
    except Exception as e:
        print(f'error downloading page {i} with err {e}')
        continue
    
    page_name = link.split('/')[-2]
    with open(f'raw/pages/{page_name}', 'w') as f:
        f.write(source)
    
    try:
        content = Moyka78Page(source)
        records.append(content)
    except Exception as e:
        print(f'error parsing page {i} with err {e}')
        continue

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████| 357/357 [01:56<00:00,  3.07it/s]


In [5]:
make_tuple = lambda p: (p.get_header(), p.get_main_text() + '\n' + p.get_citation(), p.get_create_date())
tuples = [make_tuple(page) for page in records]

df = pd.DataFrame.from_records(tuples, columns=['title', 'text', 'create_dttm']).assign(link=links)
print(df.shape)
df.head()

(357, 4)


,title,text,create_dttm,link
0,Инвалидам перекрыли входы в кафе и магазины Кр...,\n Магазины и кафе Кронштадта оказались недост...,2018-01-23T13:16,https://moika78.ru/news/2018-01-23/12164-inval...
1,Петербургские инвалиды не хотят хвалить метроп...,Оказывать помощь некоторым группам инвалидам н...,2018-01-30T15:53,https://moika78.ru/news/2018-01-30/13170-peter...
2,Петербургских родителей детей-инвалидов поддер...,\n Горячая линия для родителей детей с инвали...,2018-01-31T13:12,https://moika78.ru/news/2018-01-31/13312-peter...
3,В Петербурге оштрафовали «Мираж Синема» за отк...,Роспотребнадзор по Санкт-Петербургу оштрафовал...,2018-02-14T13:38,https://moika78.ru/news/2018-02-14/14985-v-pet...
4,Люди каменного века относились к инвалидам с у...,Первобытные люди жившие на территории современ...,2018-02-16T20:24,https://moika78.ru/news/2018-02-16/15298-lyudi...


In [6]:
df.to_csv('./raw/articles/moika78.csv', index=False)